# env setting

In [1]:
import sys

from numpy.core.fromnumeric import mean
sys.path.append("/Users/huangguanlun/pyProjects/quotoFuture/")
sys.path.append("/Users/huangguanlun/pyProjects/quotoFuture/qfVenv/lib/python3.8/site-packages/")

In [2]:
import simContainer
import numpy as np
from scipy.linalg import cholesky, inv

In [3]:
FLOAT_FORMATTER="{:.5f}".format
np.set_printoptions(formatter={'float_kind':FLOAT_FORMATTER})

# parameter and container declaration

In [4]:
S0Arr = [100, 0.5]
riskFreeRate = 0.03
timePeriod = 0.5
timePartitionCnt = 500
qArr = [0, 0]
sigmaArr = [0.03, 0.05]
corrMatrx = [[1, 0.4], [0.4, 1]]
simCnt = 10000
repeatCnt = 10

In [5]:
test = simContainer.pathSim(
    S0Arr= S0Arr,
    riskFreeRate= riskFreeRate,
    timePartitionCnt= timePartitionCnt,
    timePeriod= timePeriod,
    qArr= qArr,
    sigmaArr= sigmaArr,
    corrMatrx= corrMatrx,
    simCnt= simCnt,
    repeatCnt= repeatCnt
)

In [6]:
test.Z0Container.container.shape

(10, 10000, 500, 2)

# path simulation

In [7]:
test.var_cov()
test.muShift()
test.lnS0Append()
test.cumsum()

# carry cost computation

In [8]:
posValSeries = test.stSeries[..., 0] / test.stSeries[..., 1]
plSeries = np.diff(posValSeries, axis = 2)
deltaTCnt = np.ones_like(plSeries) * np.arange(plSeries.shape[2]) * (-1)
extraCarryCostSeries= plSeries * np.exp(riskFreeRate * deltaTCnt)
extraCarryCost = extraCarryCostSeries.sum(axis = 2)

In [9]:
meanExtraCarryCost = extraCarryCost.mean(axis = 1).mean(axis = 0)

In [24]:
meanExtraCarryCost

0.007184263475867617

In [17]:
initCost = (S0Arr[0] / S0Arr[1])

In [25]:
initCost

200.0

In [18]:
simAns = meanExtraCarryCost + initCost

In [27]:
var_T = test.sigmaArr * np.power(test.timePeriod, 0.5).reshape(1, -1)
var_T

array([[0.02121, 0.03536]])

In [29]:
covMtrx = test.corrMtrx * np.tensordot(var_T, var_T, axes= (0, 0))
covMtrx

array([[0.00045, 0.00030],
       [0.00030, 0.00125]])

In [30]:
formulaSol = S0Arr[0] * np.exp(riskFreeRate * timePeriod) * np.exp(covMtrx[0][1])

In [31]:
stdErr = extraCarryCost.mean(axis = 1).std()

In [32]:
stdErr

0.00969931510873078

In [22]:
formulaSol / S0Arr[1], simAns

(203.02273473674808, 200.00718426347586)

In [33]:
simAns - 2 * stdErr, formulaSol / S0Arr[1], simAns + 2 * stdErr

(199.9877856332584, 203.08352884395197, 200.02658289369333)

In [57]:
formulaSol, simAns * S0Arr[1]

(101.51155008899975, 101.52773522813517)